# Análise comparativa de modelos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown as md
%matplotlib inline

from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("../data/raw/Churn_Modelling.csv")
df_dict = pd.read_csv("../data/external/dicionario.csv")

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

## 1 - Preparação de dados

In [7]:
#Dataset
display(md("### `DataSet`"))
display(df.head(10))

#Print Dicionario
display(md("### `Dicionário de Dados`"))
display(df_dict)


dado_faltante = df.isnull().any(axis=1).sum()
if dado_faltante != 0:
    display(md("---"))
    display(md(f"Existem `{dado_faltante}` dados faltantes no Dataset."))
    display(md("---"))
else:
    print("Não existe dado faltante no Dataset.")




# Normalização

### `DataSet`

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1,1,101348.88,1
1,2.0,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0,1,112542.58,0
2,3.0,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1,0,113931.57,1
3,4.0,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0,0,93826.63,0
4,5.0,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1,1,79084.10,0
5,6.0,15574012.0,Chu,645.0,Spain,Male,44.0,8.0,113755.78,2.0,1,0,149756.71,1
6,7.0,15592531.0,Bartlett,822.0,France,Male,50.0,7.0,0.00,2.0,1,1,10062.80,0
7,8.0,15656148.0,Obinna,376.0,Germany,Female,29.0,4.0,115046.74,4.0,1,0,119346.88,1
8,9.0,15792365.0,He,501.0,France,Male,44.0,4.0,142051.07,2.0,0,1,74940.50,0
9,10.0,15592389.0,H?,684.0,France,Male,27.0,2.0,134603.88,1.0,1,1,71725.73,0


### `Dicionário de Dados`

,Variavel,Descrição,Tipo,Subtipo
0,RowNumber,O número sequencial atribuído a cada linha no ...,Quantitativo,Discreta
1,CustomerId,Identificador único de cada cliente,Quantitativo,Discreta
2,Surname,Indica o sobrenome do cliente.,Qualitativo,Nominal
3,CreditScore,Indica a pontuação de crédito do cliente.,Quantitativo,Discreta
4,Geography,Localização geográfica do cliente (país ou reg...,Qualitativo,Nominal
5,Gender,Gênero do cliente,Qualitativo,Nominal
6,Age,Idade do cliente,Quantitativo,Discreta
7,Tenure,Indica número de anos que o cliente está no ba...,Quantitativo,Discreta
8,Balance,Indica o saldo da conta do cliente,Quantitativo,Contínua
9,NumOfProducts,Indica o número de produtos bancários que o cl...,Quantitativo,Discreta


Não existe dado faltante no Dataset.


### `Tratamento de dados faltantes`

In [4]:
# Tratamento de dados faltantes

#Tratamento Quantitativo
var_quantitativa = df_dict.query("Tipo == 'Quantitativo'").Variavel.to_list()
#Imputando mediana nas variáveis Quantitativas
imputer = SimpleImputer(strategy="median")
imputer.fit(df[var_quantitativa])
df[var_quantitativa] = imputer.transform(df[var_quantitativa])

    
# Tratamento Qualitativo
coluna_predicao = 'Exited'

var_nominal= df_dict.query("Subtipo == 'Nominal' & Variavel != @coluna_predicao").Variavel.to_list()

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(df[var_nominal])
df[var_nominal] = imputer.transform(df[var_nominal])

### `Tratamento de dados discrepantes`

In [9]:
# Tratamento de dados discrepantes

# Definindo limites para os outliers
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
display(md("--- \n#### **Limites para anamolias**"))
print(f"Limite inferior: {lower_bound}")
print(f"Limite superior: {upper_bound}")
display(md("---"))
# Aplicando a modificação diretamente no DataFrame
df['Age'] = df['Age'].apply(lambda age: upper_bound if age > upper_bound else (lower_bound if age < lower_bound else age))
# Removendo as linhas onde a idade é menor que 18
df = df[df['Age'] >= 18]

# Exibindo o DataFrame atualizado
display(md("#### **Transformando idades discrepantes em idades limites**"))
df



--- 
#### **Limites para anamolias**

Limite inferior: 14.0
Limite superior: 62.0


---

#### **Transformando idades discrepantes em idades limites**

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,15634602.0,Hargrave,619.0,France,Female,42.0,2.0,0.00,1.0,1,1,101348.88,1
1,2.0,15647311.0,Hill,608.0,Spain,Female,41.0,1.0,83807.86,1.0,0,1,112542.58,0
2,3.0,15619304.0,Onio,502.0,France,Female,42.0,8.0,159660.80,3.0,1,0,113931.57,1
3,4.0,15701354.0,Boni,699.0,France,Female,39.0,1.0,0.00,2.0,0,0,93826.63,0
4,5.0,15737888.0,Mitchell,850.0,Spain,Female,43.0,2.0,125510.82,1.0,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996.0,15606229.0,Obijiaku,771.0,France,Male,39.0,5.0,0.00,2.0,1,0,96270.64,0
9996,9997.0,15569892.0,Johnstone,516.0,France,Male,35.0,10.0,57369.61,1.0,1,1,101699.77,0
9997,9998.0,15584532.0,Liu,709.0,France,Female,36.0,7.0,0.00,1.0,0,1,42085.58,1
9998,9999.0,15682355.0,Sabbatini,772.0,Germany,Male,42.0,3.0,75075.31,2.0,1,0,92888.52,1


### `Codificação de variáveis`